In [1]:
# 引入依赖
import cracknuts as cn

# 创建 cracker 
cracker = cn.new_cracker('192.168.0.15')

In [2]:
cracker.connect(force_update_bin=True, force_write_default_config=True)

In [5]:
import random

In [9]:

cracker.nut_voltage_enable()
cracker.nut_voltage(3.3)


(0, None)

In [11]:
cracker.uart_io_enable()

(0, None)

In [13]:
cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

# plaintext_data = random.randbytes(aes_data_len)
# tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
cmd = cmd_set_aes_enc_key + ' ' + aes_key
status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

print(cmd)
print(status)
print(ret.hex(' ') if ret else 'None')

# cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)

01 00 00 00 00 00 00 10 11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff
0
00 00 00 00 00 00


In [17]:
cmd_aes_enc = "01 02 00 00 00 00 00 10"
plaintext_data = random.randbytes(aes_data_len)
tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
print(tx_data.hex(' '))
# s, r = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
# s, r = cracker.uart_transmit_receive(tx_data, rx_count=6, is_trigger=True)
s, r = cracker.uart_transmit(tx_data, is_trigger=True)

print(r.hex(' ') if r else 'None')

01 02 00 00 00 00 00 10 af 17 2c 66 07 0a f4 52 f5 92 de dc 7b fe 5c 85
None


In [16]:
s, r = cracker.uart_receive_fifo_dump()
print(r.hex(' ') if r else 'None')

None


In [18]:
s, r = cracker.uart_receive(rx_count=22)
print(r.hex(' ') if r else 'None')

[WARNING] 2025-11-13 20:26:51,107 cracker_basic.send_and_receive:525 Command 0x0263 with payload 00 16 00 00 00 27 10 received a non-OK response with status code 0x0002, and payload 00 00 00 00 00 10 cf f3 02 b8 bd b2 8a c5 76 ef 38 04 bf 6d 92 f5.


00 00 00 00 00 10 cf f3 02 b8 bd b2 8a c5 76 ef 38 04 bf 6d 92 f5


In [ ]:
print(cracker.register_read(base_address=0x43c10000, offset=0x54))
print(cracker.register_read(base_address=0x43c10000, offset=0x58))


In [ ]:
cracker.get_id() # 获取设备ID

In [ ]:
import random
import time
from cracknuts.cracker import serial


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(c):
    cracker.nut_voltage_enable()
    cracker.nut_voltage(3.3)
    cracker.nut_clock_enable()
    cracker.nut_clock_freq('8M')
    cracker.uart_io_enable()
    cracker.osc_sample_clock('48m')
    cracker.osc_sample_length(sample_length)
    cracker.osc_trigger_source('N')
    cracker.osc_analog_gain('B', 10)
    cracker.osc_trigger_level(0)
    cracker.osc_trigger_mode('E')
    cracker.osc_trigger_edge('U')
    cracker.uart_config(baudrate=serial.Baudrate.BAUDRATE_115200, bytesize=serial.Bytesize.EIGHTBITS, parity=serial.Parity.PARITY_NONE, stopbits=serial.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(cracer, count):
    plaintext_data = random.randbytes(aes_data_len)
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    
    return {
        "plaintext": plaintext_data,
        "ciphertext": ret[-aes_data_len:],
        "key": bytes.fromhex(aes_key)
    }


def finish(c):
    ...
    # print('optional behavior')


acq = cn.new_acquisition(cracker, init=init, do=do, finish=finish)

In [ ]:
cn.panel(acq)

In [ ]:
cracker.set_logging_level('debug')

In [ ]:
cracker.get_firmware_version()